# Module 06: Production Deployment - Practice Notebook

**Level:** Advanced  
**Duration:** 4-5 hours  
**Updated:** December 2025 - Real deployment patterns

## Learning Objectives

Production deployment mastery:
- ✅ LangSmith monitoring and tracing
- ✅ Deployment configuration (Platform, BYOC, Self-hosted)
- ✅ Security and authentication patterns
- ✅ Load testing and performance
- ✅ Production best practices


In [ ]:
# Setup
%pip install -q -U langgraph langchain langchain-openai langsmith python-dotenv

import os
from dotenv import load_dotenv
load_dotenv()

print('✅ Deployment tools ready!')


---

## Exercise 1: LangSmith Monitoring 🎯

**Objective:** Enable production monitoring and observability.


In [ ]:
# Exercise 1: Enable LangSmith tracing

os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY', 'your-key-here')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'production-agents'

from langsmith import traceable
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

class MonitoredState(TypedDict):
    input: str
    output: str

@traceable(name='critical_node', tags=['production', 'monitored'])
def monitored_node(state: MonitoredState):
    # This node is automatically traced to LangSmith
    result = f'Processed: {state["input"]}'
    return {'output': result}

workflow = StateGraph(MonitoredState)
workflow.add_node('process', monitored_node)
workflow.add_edge(START, 'process')
workflow.add_edge('process', END)
app = workflow.compile()

# Test - view trace at https://smith.langchain.com
result = app.invoke({'input': 'test data'})
print(f'Result: {result}')
print('✅ Check LangSmith dashboard for trace!')


---

## Exercise 2: Deployment Configuration 🎯

**Objective:** Create deployment configs for different environments.


In [ ]:
# Exercise 2: Create langgraph.json for Platform deployment

import json

# LangGraph Platform configuration
platform_config = {
    'dependencies': [
        'langgraph',
        'langchain-openai',
        'python-dotenv'
    ],
    'graphs': {
        'my_agent': './src/agent.py:graph'
    },
    'env': {
        'OPENAI_API_KEY': '$OPENAI_API_KEY',
        'LANGSMITH_API_KEY': '$LANGSMITH_API_KEY'
    }
}

# Kubernetes Helm values
helm_values = {
    'replicaCount': 3,
    'image': {
        'repository': 'my-org/my-agent',
        'tag': 'v1.0.0'
    },
    'resources': {
        'requests': {'memory': '2Gi', 'cpu': '1000m'},
        'limits': {'memory': '4Gi', 'cpu': '2000m'}
    },
    'autoscaling': {
        'enabled': True,
        'minReplicas': 3,
        'maxReplicas': 10,
        'targetCPUUtilizationPercentage': 70
    }
}

print('Platform config:', json.dumps(platform_config, indent=2))
print('\nHelm values:', json.dumps(helm_values, indent=2))


---

## Exercise 3: Security Implementation 🎯

**Objective:** Add authentication and secure secrets management.


In [ ]:
# Exercise 3: Security patterns

from fastapi import FastAPI, Depends, HTTPException, Security
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
import jwt

app_fastapi = FastAPI()
security = HTTPBearer()

SECRET_KEY = os.getenv('JWT_SECRET_KEY', 'your-secret-key')

def verify_token(credentials: HTTPAuthorizationCredentials = Security(security)):
    token = credentials.credentials
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
        return payload
    except jwt.InvalidTokenError:
        raise HTTPException(status_code=401, detail='Invalid token')

@app_fastapi.post('/invoke')
async def invoke_agent(request: dict, auth = Depends(verify_token)):
    # Only authenticated requests reach here
    user_id = auth.get('user_id')
    result = app.invoke(request)
    return {'user_id': user_id, 'result': result}

print('✅ Secure API with JWT authentication')


---

## Exercise 4: Load Testing 🎯

**Objective:** Test agent performance under load.


In [ ]:
# Exercise 4: Performance testing

import asyncio
import time

async def load_test(num_requests=100):
    start = time.time()
    
    async def single_request():
        # Simulate request
        return app.invoke({'input': 'test'})
    
    # Run requests concurrently
    tasks = [single_request() for _ in range(num_requests)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    duration = time.time() - start
    successful = sum(1 for r in results if not isinstance(r, Exception))
    
    print(f'📊 Load Test Results:')
    print(f'  Total requests: {num_requests}')
    print(f'  Successful: {successful}')
    print(f'  Failed: {num_requests - successful}')
    print(f'  Duration: {duration:.2f}s')
    print(f'  Throughput: {num_requests/duration:.2f} req/s')
    print(f'  Avg latency: {(duration/num_requests)*1000:.0f}ms')

# Run test
await load_test(100)


---

## Exercise 5: Production Checklist 🎯

**Objective:** Validate production readiness.


In [ ]:
# Exercise 5: Production readiness checklist

checklist = {
    'Monitoring': {
        'LangSmith tracing': os.getenv('LANGCHAIN_TRACING_V2') == 'true',
        'Error tracking': True,  # Implement your check
        'Performance metrics': True
    },
    'Security': {
        'Authentication': True,
        'Secrets encrypted': bool(os.getenv('JWT_SECRET_KEY')),
        'Network policies': True
    },
    'Scalability': {
        'Auto-scaling configured': True,
        'Load tested': True,
        'Checkpointer persistent': True
    },
    'Reliability': {
        'Error handling': True,
        'Retry logic': True,
        'Circuit breakers': False  # TODO: implement
    }
}

for category, checks in checklist.items():
    print(f'\n{category}:')
    for check, status in checks.items():
        icon = '✅' if status else '❌'
        print(f'  {icon} {check}')

all_passed = all(all(checks.values()) for checks in checklist.values())
print(f'\n{"✅ PRODUCTION READY" if all_passed else "⚠️ NEEDS WORK"}')


---

## 📚 Summary

Production deployment skills:
- ✅ LangSmith monitoring
- ✅ Deployment configurations
- ✅ Security with JWT
- ✅ Load testing
- ✅ Production checklist

**Next:** Module 07 - Advanced Patterns! 🚀
